In [2]:
import torch_xla.runtime
import torch
import torch_xla

x = torch.randn(2, 3, 4, 5, requires_grad=True)
y = torch.randn(2, 5, 6, requires_grad=True)

with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    x = x.to('xla').requires_grad_()
    y = y.to('xla').requires_grad_()
    out = torch.einsum("ebcm,emh->ebch", x, y)

print(torch_xla._XLAC._get_xla_tensors_text([out]))
print(torch_xla._XLAC._get_xla_tensors_hlo([out]))

/home/bbahl/miniconda3/envs/torchprime/lib/python3.10/site-packages/torch_xla/__init__.py:251: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


IR {
  %0 = f32[2,5,6]{2,1,0} xla::device_data(), xla_shape=f32[2,5,6]{2,1,0}
  %1 = f32[2,3,4,5]{3,2,1,0} xla::device_data(), xla_shape=f32[2,3,4,5]{3,2,1,0}
  %2 = f32[2,3,4,6]{3,2,1,0} aten::einsum(%1, %0), xla_shape=f32[2,3,4,6]{3,2,1,0}, ROOT=0
}

HloModule IrToHlo.6, entry_computation_layout={(f32[2,5,6]{2,1,0}, f32[2,3,4,5]{3,2,1,0})->(f32[2,3,4,6]{3,2,1,0})}

ENTRY %IrToHlo.6 (p0.1: f32[2,5,6], p1.2: f32[2,3,4,5]) -> (f32[2,3,4,6]) {
  %p1.2 = f32[2,3,4,5]{3,2,1,0} parameter(1)
  %p0.1 = f32[2,5,6]{2,1,0} parameter(0)
  %dot.3 = f32[2,3,4,6]{3,2,1,0} dot(f32[2,3,4,5]{3,2,1,0} %p1.2, f32[2,5,6]{2,1,0} %p0.1), lhs_batch_dims={0}, lhs_contracting_dims={3}, rhs_batch_dims={0}, rhs_contracting_dims={1}, frontend_attributes={grad_x="false",grad_y="false"}
  %transpose.4 = f32[2,3,4,6]{3,2,1,0} transpose(f32[2,3,4,6]{3,2,1,0} %dot.3), dimensions={0,1,2,3}
  ROOT %tuple.5 = (f32[2,3,4,6]{3,2,1,0}) tuple(f32[2,3,4,6]{3,2,1,0} %transpose.4)
}




In [8]:
device = torch_xla.device()
x1 = torch.randn(2, 5, 6, requires_grad=True)
y1 = torch.randn(2, 5, 4, 8, requires_grad=True)

with torch.enable_grad():
  x1 = x1.to('xla')
  y1 = y1.to('xla')
  out1 = torch.einsum("bsm,bsec->bmec", x1, y1)

print(torch_xla._XLAC._get_xla_tensors_text([out1]))
print(torch_xla._XLAC._get_xla_tensors_hlo([out1]))

IR {
  %0 = f32[2,5,4,8]{3,1,2,0} xla::device_data(), xla_shape=f32[2,5,4,8]{3,1,2,0}
  %1 = f32[2,5,6]{2,1,0} xla::device_data(), xla_shape=f32[2,5,6]{2,1,0}
  %2 = f32[2,6,4,8]{3,2,1,0} aten::einsum(%1, %0), xla_shape=f32[2,6,4,8]{3,2,1,0}, ROOT=0
}

HloModule IrToHlo.6, entry_computation_layout={(f32[2,5,4,8]{3,1,2,0}, f32[2,5,6]{2,1,0})->(f32[2,6,4,8]{3,2,1,0})}

ENTRY %IrToHlo.6 (p0.1: f32[2,5,4,8], p1.2: f32[2,5,6]) -> (f32[2,6,4,8]) {
  %p1.2 = f32[2,5,6]{2,1,0} parameter(1)
  %p0.1 = f32[2,5,4,8]{3,1,2,0} parameter(0)
  %dot.3 = f32[2,6,4,8]{3,2,1,0} dot(f32[2,5,6]{2,1,0} %p1.2, f32[2,5,4,8]{3,1,2,0} %p0.1), lhs_batch_dims={0}, lhs_contracting_dims={1}, rhs_batch_dims={0}, rhs_contracting_dims={1}, frontend_attributes={grad_x="false",grad_y="false"}
  %transpose.4 = f32[2,6,4,8]{3,2,1,0} transpose(f32[2,6,4,8]{3,2,1,0} %dot.3), dimensions={0,1,2,3}
  ROOT %tuple.5 = (f32[2,6,4,8]{3,2,1,0}) tuple(f32[2,6,4,8]{3,2,1,0} %transpose.4)
}




In [7]:
x2 = torch.randn(2, 5, 4, 8, requires_grad=True)
y2 = torch.randn(4, 5, 6, requires_grad=True)

with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    x2 = x2.to('xla').requires_grad_()
    y2 = y2.to('xla').requires_grad_()
    out2= torch.einsum("bmec,emh->ebch", x2, y2)

print(torch_xla._XLAC._get_xla_tensors_text([out2]))
print(torch_xla._XLAC._get_xla_tensors_hlo([out2]))

IR {
  %0 = f32[4,5,6]{2,1,0} xla::device_data(), xla_shape=f32[4,5,6]{2,1,0}
  %1 = f32[2,5,4,8]{3,1,2,0} xla::device_data(), xla_shape=f32[2,5,4,8]{3,1,2,0}
  %2 = f32[4,2,8,6]{3,2,1,0} aten::einsum(%1, %0), xla_shape=f32[4,2,8,6]{3,2,1,0}, ROOT=0
}

HloModule IrToHlo.6, entry_computation_layout={(f32[4,5,6]{2,1,0}, f32[2,5,4,8]{3,1,2,0})->(f32[4,2,8,6]{3,2,1,0})}

ENTRY %IrToHlo.6 (p0.1: f32[4,5,6], p1.2: f32[2,5,4,8]) -> (f32[4,2,8,6]) {
  %p1.2 = f32[2,5,4,8]{3,1,2,0} parameter(1)
  %p0.1 = f32[4,5,6]{2,1,0} parameter(0)
  %dot.3 = f32[4,2,8,6]{3,2,1,0} dot(f32[2,5,4,8]{3,1,2,0} %p1.2, f32[4,5,6]{2,1,0} %p0.1), lhs_batch_dims={2}, lhs_contracting_dims={1}, rhs_batch_dims={0}, rhs_contracting_dims={1}, frontend_attributes={grad_x="false",grad_y="false"}
  %transpose.4 = f32[4,2,8,6]{3,2,1,0} transpose(f32[4,2,8,6]{3,2,1,0} %dot.3), dimensions={0,1,2,3}
  ROOT %tuple.5 = (f32[4,2,8,6]{3,2,1,0}) tuple(f32[4,2,8,6]{3,2,1,0} %transpose.4)
}




In [4]:
x3 = torch.randn(2, 5, 4, 8, requires_grad=True)
y3 = torch.randn(2, 8, 6, requires_grad=True)

with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    x3 = x3.to('xla').requires_grad_()
    y3 = y3.to('xla').requires_grad_()
    out3= torch.einsum("ebch,ehm->ebcm", x3, y3)

print(torch_xla._XLAC._get_xla_tensors_text([out3]))
print(torch_xla._XLAC._get_xla_tensors_hlo([out3]))

IR {
  %0 = f32[2,8,6]{1,2,0} xla::device_data(), xla_shape=f32[2,8,6]{1,2,0}
  %1 = f32[2,5,4,8]{3,1,2,0} xla::device_data(), xla_shape=f32[2,5,4,8]{3,1,2,0}
  %2 = f32[2,5,4,6]{3,2,1,0} aten::einsum(%1, %0), xla_shape=f32[2,5,4,6]{3,2,1,0}, ROOT=0
}

HloModule IrToHlo.6, entry_computation_layout={(f32[2,8,6]{1,2,0}, f32[2,5,4,8]{3,1,2,0})->(f32[2,5,4,6]{3,2,1,0})}

ENTRY %IrToHlo.6 (p0.1: f32[2,8,6], p1.2: f32[2,5,4,8]) -> (f32[2,5,4,6]) {
  %p1.2 = f32[2,5,4,8]{3,1,2,0} parameter(1)
  %p0.1 = f32[2,8,6]{1,2,0} parameter(0)
  %dot.3 = f32[2,5,4,6]{3,2,1,0} dot(f32[2,5,4,8]{3,1,2,0} %p1.2, f32[2,8,6]{1,2,0} %p0.1), lhs_batch_dims={0}, lhs_contracting_dims={3}, rhs_batch_dims={0}, rhs_contracting_dims={1}, frontend_attributes={grad_x="false",grad_y="false"}
  %transpose.4 = f32[2,5,4,6]{3,2,1,0} transpose(f32[2,5,4,6]{3,2,1,0} %dot.3), dimensions={0,1,2,3}
  ROOT %tuple.5 = (f32[2,5,4,6]{3,2,1,0}) tuple(f32[2,5,4,6]{3,2,1,0} %transpose.4)
}




In [5]:
x4 = torch.randn(2, 5, 4, 8, requires_grad=True)
y4 = torch.randn(5, 6 , 2, 4, requires_grad=True)

with torch.enable_grad():
  with torch_xla.runtime.xla_device():
    x4 = x4.to('xla').requires_grad_()
    y4 = y4.to('xla').requires_grad_()
    out4= torch.einsum("ebcm,bsec->bsm", x4, y4)

print(torch_xla._XLAC._get_xla_tensors_text([out4]))
print(torch_xla._XLAC._get_xla_tensors_hlo([out4]))

IR {
  %0 = f32[5,6,2,4]{1,0,3,2} xla::device_data(), xla_shape=f32[5,6,2,4]{1,0,3,2}
  %1 = f32[2,5,4,8]{3,1,2,0} xla::device_data(), xla_shape=f32[2,5,4,8]{3,1,2,0}
  %2 = f32[5,6,8]{1,2,0} aten::einsum(%1, %0), xla_shape=f32[5,6,8]{1,2,0}, ROOT=0
}

HloModule IrToHlo.6, entry_computation_layout={(f32[5,6,2,4]{1,0,3,2}, f32[2,5,4,8]{3,1,2,0})->(f32[5,6,8]{1,2,0})}

ENTRY %IrToHlo.6 (p0.1: f32[5,6,2,4], p1.2: f32[2,5,4,8]) -> (f32[5,6,8]) {
  %p1.2 = f32[2,5,4,8]{3,1,2,0} parameter(1)
  %p0.1 = f32[5,6,2,4]{1,0,3,2} parameter(0)
  %dot.3 = f32[5,8,6]{2,1,0} dot(f32[2,5,4,8]{3,1,2,0} %p1.2, f32[5,6,2,4]{1,0,3,2} %p0.1), lhs_batch_dims={1}, lhs_contracting_dims={0,2}, rhs_batch_dims={0}, rhs_contracting_dims={2,3}, frontend_attributes={grad_x="false",grad_y="false"}
  %transpose.4 = f32[5,6,8]{1,2,0} transpose(f32[5,8,6]{2,1,0} %dot.3), dimensions={0,2,1}
  ROOT %tuple.5 = (f32[5,6,8]{1,2,0}) tuple(f32[5,6,8]{1,2,0} %transpose.4)
}


